mport Essential dependencies

In [1]:
import streamlit as sl
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

unction to load the vectordatabase

In [2]:
def load_knowledgeBase():
        embeddings=OpenAIEmbeddings(api_key=os.environ['API_key'] )
        DB_FAISS_PATH = '../vectorstore'
        db = FAISS.load_local(DB_FAISS_PATH, embeddings)
        return db
        
#function to load the OPENAI LLM
def load_llm():
        from langchain_openai import ChatOpenAI
        llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, api_key=os.environ['API_key'] )
        return llm

reating prompt template using langchain

In [3]:
def load_prompt():
        prompt = """ You need to answer the question in the sentence as same as in the  pdf content. . 
        Given below is the context and question of the user.
        context = {context}
        question = {question}
        if the answer is not in the pdf answer "Sorry, I'm not sure how to respond to this"
         """
        prompt = ChatPromptTemplate.from_template(prompt)
        return prompt

In [4]:
def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

In [5]:
import os

if __name__=='__main__':
        sl.header("welcome to the 📝PDF bot")
        sl.write("Please enter your API key: ")
        os.environ['API_key'] = sl.text_input('Enter API key')
        sl.write("🤖 You can chat by Entering your queries ")
        knowledgeBase=load_knowledgeBase()
        llm=load_llm()
        prompt=load_prompt()
        
        query=sl.text_input('Enter some text')
        
        
        if(query):
                #getting only the chunks that are similar to the query for llm to produce the output
                similar_embeddings=knowledgeBase.similarity_search(query)
                similar_embeddings=FAISS.from_documents(documents=similar_embeddings, embedding=OpenAIEmbeddings(api_key=os.environ['API_key'] ))
                
                #creating the chain for integrating llm,prompt,stroutputparser
                retriever = similar_embeddings.as_retriever()
                rag_chain = (
                        {"context": retriever | format_docs, "question": RunnablePassthrough()}
                        | prompt
                        | llm
                        | StrOutputParser()
                    )
                
                response=rag_chain.invoke(query)
                sl.write(response)
        

2024-07-09 19:50:51.427 
  command:

    streamlit run /home/ubuntu/.local/lib/python3.8/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-07-09 19:50:51.429 Session state does not function when running a script without `streamlit run`


ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)